In [51]:
import numpy as np
import pandas as pd
import time

import requests
import bs4
import json
import re

In [52]:
def stock_prices(ticker):
    """
    ticker is the abbreviated symbol for a stock e.g.AAPL
    this function returns the daily price history of the requested stock as a dataframe
    """
    stock_endpoint = 'https://financialmodelingprep.com/api/v3/historical-price-full/'
    response = requests.get(stock_endpoint + ticker + '?apikey=70407133ea11d7284c70bbca4eee2547').json()
    type(response) == dict
    return pd.DataFrame(response['historical'])

def ny_times_articles(keyword):
    url_dict = {}
    index = 0
    for i in range(10):
        response = requests.get("https://api.nytimes.com/svc/search/v2/articlesearch.json?q=apple&fq=news_desk:Business&page="+str(i)+"&api-key=fO0tDSRQQdU68GkuXbMjt1uA2FYImzVp").json()
        try:
            docs = response['response']['docs']
            for item in docs:
                if item['web_url'][8] == 'w': # To remove 'https://bits.blogs.nytimes...' or ''https://dealbook.nytimes...'
                    link = item['web_url']
                    date = link[24:28] + '-' + link[29:31] + '-' + link[32:34]
                    url_dict[index] = tuple((link, date))
                    index += 1
        except KeyError:
            continue
    return url_dict

In [53]:
test_dict = ny_times_articles("Apple")

print(test_dict)

{0: ('https://www.nytimes.com/2020/12/16/technology/facebook-takes-the-gloves-off-in-feud-with-apple.html', '2020-12-16'), 1: ('https://www.nytimes.com/2020/12/13/business/media/apple-gawker-tim-cook.html', '2020-12-13'), 2: ('https://www.nytimes.com/2020/12/23/business/dealbook/trump-stimulus-veto.html', '2020-12-23'), 3: ('https://www.nytimes.com/2020/12/01/technology/amazon-apple-chips-intel-arm.html', '2020-12-01'), 4: ('https://www.nytimes.com/2020/12/17/technology/google-antitrust-monopoly.html', '2020-12-17'), 5: ('https://www.nytimes.com/2020/12/17/business/dealbook/tech-apple-facebook-fight.html', '2020-12-17'), 6: ('https://www.nytimes.com/2020/12/15/technology/big-tech-regulation-europe.html', '2020-12-15'), 7: ('https://www.nytimes.com/2020/12/14/technology/big-tech-lobbying-europe.html', '2020-12-14'), 8: ('https://www.nytimes.com/2020/12/09/technology/personaltech/amazon-halo-review.html', '2020-12-09'), 9: ('https://www.nytimes.com/2020/11/18/technology/apple-app-store-f

In [54]:
stock_prices('AAPL')

,date,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime
0,2020-12-31,134.080002,134.740005,131.720001,132.690002,132.690002,98990400.0,98990400.0,-1.39000,-1.037,133.05000,"December 31, 20",-0.01037
1,2020-12-30,135.580002,135.990005,133.399994,133.720001,133.720001,96452100.0,96452100.0,-1.86000,-1.372,134.37000,"December 30, 20",-0.01372
2,2020-12-29,138.050003,138.789993,134.339996,134.869995,134.869995,121047300.0,121047300.0,-3.18001,-2.304,135.99999,"December 29, 20",-0.02304
3,2020-12-28,133.990005,137.339996,133.509995,136.690002,136.690002,124486200.0,124486200.0,2.70000,2.015,135.84666,"December 28, 20",0.02015
4,2020-12-24,131.320007,133.460007,131.100006,131.970001,131.970001,54930100.0,54930100.0,0.64999,0.495,132.17667,"December 24, 20",0.00495
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254,2016-01-08,24.637501,24.777500,24.190001,24.240000,22.275295,283192000.0,283192000.0,-0.39750,-1.613,24.40250,"January 08, 16",-0.01613
1255,2016-01-07,24.670000,25.032499,24.107500,24.112499,22.158121,324377600.0,324377600.0,-0.55750,-2.260,24.41750,"January 07, 16",-0.02260
1256,2016-01-06,25.139999,25.592501,24.967501,25.174999,23.134508,273829600.0,273829600.0,0.03500,0.139,25.24500,"January 06, 16",0.00139
1257,2016-01-05,26.437500,26.462500,25.602501,25.677500,23.596279,223164000.0,223164000.0,-0.76000,-2.875,25.91417,"January 05, 16",-0.02875


In [55]:
text = []
title = []
abstract = []

headers = {'user-agent': 'Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36'}
article = requests.get('https://www.nytimes.com/2006/07/01/business/01online.html', headers = headers)
soup = bs4.BeautifulSoup(article.content, 'html.parser')
article_text_p = soup.find_all('p', attrs={'class': 'css-axufdj evys1bk0'})
abstract_text_p = soup.find('p', attrs={'class': 'css-w6ymp8 e1wiw3jv0'})
title_text_h1 = soup.find('h1', attrs={'data-test-id': 'headline'})
temp = []
print(title_text_h1)
title.append(title_text_h1.text)
abstract.append(abstract_text_p.text)
    
for item in article_text_p:
    temp.append(item.text)
space = ' '
article_text = space.join(temp)
text.append(article_text)

<h1 class="css-1fyocly e1h9rw200" data-test-id="headline" id="link-2698b7df">Apple's Got a Secret</h1>


AttributeError: 'NoneType' object has no attribute 'text'